## Hospital Scheduling Using Bin Packing Approach

### Objective:
The objective of this approach is to efficiently schedule surgeries in hospital operating rooms while considering surgery durations, room availability, and surgery priorities.

### Method:
1. **Model Parameters:**
    - Number of surgeries, types, rooms, and days.
    - Operating hours per day.
    - Surgery durations, priorities, and total surgeries of each type.

2. **Variables:**

- $ x_{s,d,r} $: Binary decision variable representing whether surgery \( s \) is scheduled on day \( d \) in room \( r \). It takes the value 1 if the surgery is scheduled, and 0 otherwise.
- $ P_s $: Priority associated with surgery \( s \). This priority reflects the importance or urgency of performing the surgery.
- $ W_d $: Weight assigned to day \( d \). This weight decreases over time to prioritize earlier scheduling.

3. **Objective Function:**

The objective is to maximize the total late cost by assigning surgeries to days and rooms in a way that maximizes the product of surgery priority, day weight, and the binary decision variable indicating whether the surgery is scheduled on that day and in that room.

$$
\text{Maximize} \sum_{s \in \text{surgeries}} \sum_{d=1}^{\text{days}} \sum_{r=1}^{\text{rooms}} P_s \cdot W_d \cdot x_{s,d,r}
$$

This objective function aims to maximize the sum of the products of surgery priorities, day weights, and the binary decision variables representing scheduled surgeries across all surgeries, days, and rooms.



4. **Constraints:**
    - Limit the total duration of surgeries in each room on each day to the operating hours.
    - Ensure each surgery is assigned to at most one room on one day.
    - Limit the total number of surgeries of each type scheduled.
    - Add a variance based slack time at the end of each day (for each room) based on the sum of variances of surgeries scheduled.

5. **Solving the Model:**
    - Optimize the model to find the optimal surgery schedule using gurobi GRB.

6. **Output:**
    - Outputs a HTML visualization via Plotly to show the 5-day, 3-room schedule, along with missed surgery frequencies.
    - Outputs 3 data frames showing OR utilization per day, schedule of surgeries, missed surgery frequencies.


In [1]:
# Surgery type attributes (previously defined)
surgery_type_attributes = {
    "Cardiovascular": {"duration": 4, "variance": 1.0, "priority": 5},
    "Neurological": {"duration": 2, "variance": 1.2, "priority": 4},
    "Orthopedic": {"duration": 3, "variance": 0.5, "priority": 3},
    "Gastrointestinal": {"duration": 4, "variance": 0.7, "priority": 2},
    "Cosmetic": {"duration": 5, "variance": 0.3, "priority": 1},
}

total_surgeries_of_each_type = {1: 99, 2: 99, 3: 99, 4: 99, 5: 99}

type_names = {
    1: "Cardiovascular",
    2: "Neurological",
    3: "Orthopedic",
    4: "Gastrointestinal",
    5: "Cosmetic",
}

# Generate the surgeries list based on the new structure
surgeries = [
    f"{type_names[type_id]} Surgery {i}"
    for type_id, count in total_surgeries_of_each_type.items()
    for i in range(1, count + 1)
]


In [2]:
from gurobipy import Model, GRB
import numpy as np

# Constants
days = 5
rooms = 3
hours_per_day = 8  # Assuming each operating room is available 8 hours per day

# Model
m = Model("SurgeryScheduling")

# Decision variables: surgery_scheduled[surgery, day, room]
surgery_scheduled = m.addVars(surgeries, range(days), range(rooms), vtype=GRB.BINARY, name="schedule")

# Define a weight for each day, decreasing over time to prioritize earlier scheduling
day_weights = {d: (days - d) for d in range(days)}

# minimize late cost, incorporating surgery priority and day weight
m.setObjective(sum(surgery_scheduled[s, d, r] * surgery_type_attributes[s.split()[0]]['priority'] * day_weights[d]
                   for s in surgeries for d in range(days) for r in range(rooms)), GRB.MAXIMIZE)


# Constraint: Each surgery can only be scheduled once
for s in surgeries:
    m.addConstr(sum(surgery_scheduled[s, d, r] for d in range(days) for r in range(rooms)) <= 1, f"OneTime_{s}")

# Constraint: Respect operating room availability (duration and operating hours)
for d in range(days):
    for r in range(rooms):
        m.addConstr(sum(surgery_scheduled[s, d, r] * surgery_type_attributes[s.split()[0]]['duration']
                        for s in surgeries) <= hours_per_day, f"RoomAvailability_{d}_{r}")

# Adjust the surgery duration constraints to incorporate pseudo-variance-based slack time
beta = 0.0  # Multiplier for variance-based slack time
for d in range(days):
    for r in range(rooms):
        slack_time_adjustment = beta * sum(
            surgery_scheduled[s, d, r] * np.sqrt(surgery_type_attributes[s.split()[0]]['variance'])
            for s in surgeries
        )
        m.addConstr(
            sum(
                surgery_scheduled[s, d, r] * surgery_type_attributes[s.split()[0]]['duration']
                for s in surgeries
            ) + slack_time_adjustment <= hours_per_day,
            f"variance_adjusted_duration_day{d+1}_room{r+1}"
        )

# Optimize model
m.optimize()

# The structure will be: surgery_name: {"day": day, "room": room}
scheduled_surgeries = {}

# Populate the dictionary based on the optimization results
for s in surgeries:
    for d in range(days):
        for r in range(rooms):
            # Check if the surgery is scheduled based on the decision variable
            if surgery_scheduled[s, d, r].X > 0.5:  # Using 0.5 as a threshold to indicate selection
                # Store the scheduled day and room for each surgery
                scheduled_surgeries[s] = {"day": d+1, "room": r+1}

# Print or use the scheduled_surgeries dictionary as needed
for surgery, info in scheduled_surgeries.items():
    print(f"{surgery} is scheduled on day {info['day']} in room {info['room']}")




Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-11
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 525 rows, 7425 columns and 22275 nonzeros
Model fingerprint: 0xa4423220
Variable types: 0 continuous, 7425 integer (7425 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 450.0000000
Presolve removed 15 rows and 0 columns
Presolve time: 0.03s
Presolved: 510 rows, 7425 columns, 14850 nonzeros
Variable types: 0 continuous, 7425 integer (7425 binary)

Root relaxation: objective 7.200000e+02, 1089 iterations, 0.02 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective

In [3]:
import pandas as pd

# Assuming scheduled_surgeries is already populated with optimization results
# Convert the scheduled_surgeries dictionary to a DataFrame
df_scheduled = pd.DataFrame([(surgery, info['day'], info['room'], surgery.split()[0])
                             for surgery, info in scheduled_surgeries.items()],
                            columns=['Surgery', 'Day', 'Room', 'Type'])

# Since 'Type' is now directly taken from the surgery name, there's no need to map it again
# However, we'll update 'Duration' based on the actual surgery type names
df_scheduled['Duration'] = df_scheduled['Type'].apply(lambda x: surgery_type_attributes[x]['duration'])

# For missed surgeries, assuming a hypothetical way to identify them,
# we will continue with an empty DataFrame as no logic for identifying missed surgeries was provided
df_missed = pd.DataFrame(columns=['Type', 'Missed Count'])

# Extract surgeries that are in the schedule
scheduled_surgeries = set(df_scheduled['Surgery'])

# Find surgeries that are not in the schedule
missed_surgeries = [surgery for surgery in surgeries if surgery not in scheduled_surgeries]

# Create DataFrame for missed surgeries
df_missed = pd.DataFrame(columns=['Surgery', 'Day', 'Room', 'Type', 'Duration'])
for surgery in missed_surgeries:
    type_name = surgery.split()[0]
    duration = surgery_type_attributes[type_name]['duration']
    # For missed surgeries, assign NaN to 'Day' and 'Room'
    df_missed = df_missed.append({'Surgery': surgery, 'Day': None, 'Room': None, 'Type': type_name, 'Duration': duration}, ignore_index=True)

# Example output of scheduled surgeries DataFrame
print(df_scheduled.head())

# Example output for missed surgeries DataFrame (based on placeholder logic, likely empty)
print(df_missed)


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
df_scheduled.head()

,Surgery,Day,Room,Type,Duration
0,Cardiovascular Surgery 1,1,3,Cardiovascular,4
1,Cardiovascular Surgery 2,2,2,Cardiovascular,4
2,Cardiovascular Surgery 3,2,2,Cardiovascular,4
3,Cardiovascular Surgery 4,2,1,Cardiovascular,4
4,Cardiovascular Surgery 5,2,1,Cardiovascular,4


In [ ]:
df_missed.head()

,Surgery,Day,Room,Type,Duration
0,Gastrointestinal Surgery 1,None,None,Gastrointestinal,4
1,Gastrointestinal Surgery 4,None,None,Gastrointestinal,4
2,Gastrointestinal Surgery 6,None,None,Gastrointestinal,4
3,Gastrointestinal Surgery 8,None,None,Gastrointestinal,4
4,Cosmetic Surgery 1,None,None,Cosmetic,5


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Assuming df_scheduled and df_missed are already defined as per previous instructions

# Initialize Figure for Operating Room Utilization
fig = go.Figure()

# Iterate through each operating room
for room in sorted(df_scheduled['Room'].unique()):
    # Group the data by day and sum the duration
    room_data = df_scheduled[df_scheduled['Room'] == room].groupby('Day').agg({'Duration': 'sum'}).reset_index()
    
    # Generate hover text showing the count of each surgery type per day
    hover_text = []
    for day in room_data['Day']:
        day_data = df_scheduled[(df_scheduled['Room'] == room) & (df_scheduled['Day'] == day)]
        type_counts = day_data['Type'].value_counts().to_dict()
        hover_text_day = ", ".join([f"{type}: {count}" for type, count in type_counts.items()])
        hover_text.append(hover_text_day)
    
    # Calculate text for the bar labels
    text = [f"Total Surgeries: {len(df_scheduled[(df_scheduled['Room'] == room) & (df_scheduled['Day'] == day)])}"
            for day in room_data['Day']]
    
    # Add bar chart for each operating room
    fig.add_trace(go.Bar(x=room_data['Day'], y=room_data['Duration'], name=f'Room {room}',
                         hoverinfo='text', text=text, hovertext=hover_text, textposition='auto'))

# Update layout for better visualization
fig.update_layout(title_text='Operating Room Utilization by Day',
                  xaxis_title="Day",
                  yaxis_title="Hours Used",
                  legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
                  barmode='group',
                  bargap=0.2)

# Add an 8-hour operating limit line and other enhancements
fig.add_hline(y=8, line_dash="dot", annotation_text="8-hour limit", annotation_position="bottom right")
fig.update_layout(title_text='Operating Room Utilization by Day',
                  xaxis_title="Day",
                  yaxis_title="Hours Used",
                  legend_title_text='Operating Room',
                  legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
                  barmode='group',
                  bargap=0.15)

# Example code to fix the DataFrame structure
df_missed = df_missed[['Type', 'Surgery', 'Duration']]
df_missed = df_missed.groupby(["Type"]).size().reset_index(name='Missed Count')

# Visualization for Missed Surgeries by Type
fig_missed = px.bar(df_missed, x='Type', y='Missed Count', text='Missed Count',
                    title='Missed Surgeries by Type',
                    labels={'Missed Count': 'Number of Missed Surgeries', 'Type': 'Surgery Type'})

# Update layout for missed surgeries chart
fig_missed.update_layout(xaxis_title="Surgery Type", yaxis_title="Number of Missed Surgeries",
                         legend_title_text='Surgery Type')

# Displaying the figures
fig.show()
fig_missed.show()


In [ ]:
# Save HTML content of both plots separately
html_content_fig = fig.to_html(full_html=False)
html_content_fig_missed = fig_missed.to_html(full_html=False)

# Combine the HTML content into one file
combined_html_content = html_content_fig + html_content_fig_missed

# Save the combined HTML content to a file
with open('operating_room_utilization_combined_3.0.html', 'w') as f:
    f.write(combined_html_content)